## 1. Instalación y Configuración

In [ ]:
# Instalar dependencias (ejecutar solo si es necesario)
# !pip install -r ../requirements.txt

In [ ]:
import sys
from pathlib import Path

# Agregar raíz al path
notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

In [ ]:
# Importar módulos principales
from src.llm.base import LLMFactory
from src.agents.touristic_agent import TouristicAgent, AgentBuilder
from src.utils.helpers import Logger
from src.utils.config import ConfigLoader

print("✓ Módulos importados correctamente")

## 2. Inicializar el Chatbot

In [ ]:
# Cargar configuración
config_loader = ConfigLoader(str(project_root / "config"))
model_config = config_loader.load_model_config()
agent_config = config_loader.load_agent_config()

print("Configuración cargada:")
print(f"  Modelo por defecto: {model_config.get('default_model')}")
print(f"  Agente: {agent_config.get('agent', {}).get('name')}")

In [ ]:
# Crear instancia del LLM
llm_provider = "openai"  # Cambiar si lo necesitas
model_settings = model_config.get("models", {}).get(llm_provider, {})

try:
    llm = LLMFactory.get_model(
        llm_provider,
        **{k: v for k, v in model_settings.items() if k != "provider"}
    )
    print(f"✓ LLM {llm_provider} inicializado correctamente")
except Exception as e:
    print(f"✗ Error: {e}")
    print(f"Asegúrate de que tienes la API key configurada")

In [ ]:
# Crear el agente
agent = AgentBuilder.create_agent(
    llm,
    agent_type="standard",
    max_iterations=agent_config.get("agent", {}).get("max_iterations", 10)
)

print(f"✓ Agente creado exitosamente")
print(f"  Herramientas disponibles: {len(agent.tools)}")

## 3. Probar Consultas

In [ ]:
# Función auxiliar para visualizar respuestas
def ask_chatbot(question):
    print(f"\n{'='*60}")
    print(f"PREGUNTA: {question}")
    print(f"{'='*60}")
    
    response = agent.process_query(question)
    
    if response["success"]:
        print(f"\nRESPUESTA:\n{response['response']}")
    else:
        print(f"\nERROR: {response.get('error', 'Desconocido')}")
        print(f"Mensaje: {response['response']}")
    
    return response

In [ ]:
# Prueba 1: Pregunta simple
response1 = ask_chatbot("¿Cuáles son las mejores atracciones para principiantes en Huaraz?")

In [ ]:
# Prueba 2: Información específica
response2 = ask_chatbot("Cuéntame detalladamente sobre la Laguna 69")

In [ ]:
# Prueba 3: Recomendación de itinerario
response3 = ask_chatbot("Tengo 3 días en Huaraz, presupuesto limitado y quiero naturaleza. ¿Qué me recomiendas?")

In [ ]:
# Prueba 4: Consulta sobre salud
response4 = ask_chatbot("Tengo miedo al mal de altura. ¿Cuáles son los síntomas y cómo prevenirlo?")

## 4. Pruebas de Base de Conocimiento

In [ ]:
from data.knowledge.huaraz_knowledge import HuarazKnowledgeBase

kb = HuarazKnowledgeBase()

# Ver todas las atracciones
print("ATRACCIONES DISPONIBLES:")
for name, attr in kb.get_all_attractions().items():
    print(f"\n  {attr.name}")
    print(f"    - Ubicación: {attr.location}")
    print(f"    - Altitud: {attr.altitude}m")
    print(f"    - Dificultad: {attr.difficulty}")
    print(f"    - Duración: {attr.duration}")

In [ ]:
# Búsqueda por dificultad
print("ATRACCIONES FÁCILES:")
easy_attractions = kb.search_by_difficulty("bajo")
for attr in easy_attractions:
    print(f"  - {attr.name} ({attr.location})")

In [ ]:
# Búsqueda por temporada
print("ATRACCIONES PARA JUNIO:")
june_attractions = kb.search_by_season("junio")
for attr in june_attractions:
    print(f"  - {attr.name}")

## 5. Pruebas de Herramientas

In [ ]:
from src.handlers.tools import (
    search_attractions,
    get_attraction_details,
    get_altitude_advice
)

# Prueba de herramienta 1
print("Búsqueda de atracciones:")
results = search_attractions("laguna")
for result in results:
    print(f"  - {result.get('name', 'N/A')}: {result.get('description', 'N/A')[:50]}...")

In [ ]:
# Prueba de herramienta 2
print("\nDetalles de atracción:")
details = get_attraction_details("Laguna Parón")
for key, value in details.items():
    print(f"  {key}: {value}")

In [ ]:
# Prueba de herramienta 3
print("\nConsejos de altitud:")
advice = get_altitude_advice()
print(f"Síntomas: {', '.join(advice['symptoms'])}")
print(f"\nPreventivos:")
for i, tip in enumerate(advice['prevention'], 1):
    print(f"  {i}. {tip}")

## 6. Análisis de Conversación

In [ ]:
# Ver historial de conversación
print("HISTORIAL DE CONVERSACIÓN:")
print(agent.get_conversation_history())

In [ ]:
# Limpiar memoria
agent.clear_memory()
print("✓ Memoria limpiada")

## 7. Experimentación Avanzada

In [ ]:
# Crear un agente especializado
expert_agent = AgentBuilder.create_agent(
    llm,
    agent_type="expert",
    max_iterations=15
)

print(f"✓ Agente experto creado")
print(f"  Iteraciones máximas: {expert_agent.max_iterations}")

In [ ]:
# Consulta compleja al agente experto
complex_query = """Soy un fotógrafo profesional interesado en capturar la naturaleza alpina.
Tengo 2 semanas disponibles en Huaraz, presupuesto alto, y estoy en excelente condición física.
Quiero evitar multitudes. ¿Qué itinerario me recomiendas?"""

response = ask_chatbot(complex_query)

## 8. Personalización de Prompts

In [ ]:
from src.prompt_engineering.prompts import PromptManager

# Ver prompt del sistema
print("PROMPT DEL SISTEMA:")
print("-" * 60)
print(PromptManager.get_system_prompt())
print("-" * 60)

## Notas y Conclusiones

- El agente usa múltiples herramientas para responder consultas
- Mantiene contexto en conversaciones
- Puede iterar si necesita información adicional
- Personalización es posible en prompts y configuración